# 【問題1】SimpleRNNのフォワードプロパゲーション実装

SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。  
今回はバッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。  
活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。  
フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。
$${a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + b\\
h_t = tanh(a_t)}$$  
${a_t}$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)  
${h_t}$ : 時刻tの状態・出力 (batch_size, n_nodes)
${x_t}$ : 時刻tの入力 (batch_size, n_features)  
${W_x}$ : 入力に対する重み (n_features, n_nodes)  
${h_{t−1}}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)  
${W_h}$ : 状態に対する重み。 (n_nodes, n_nodes)  
${b}$ : バイアス項 (1,)  
初期状態${h_0}$は全て0とすることが多いですが、任意の値を与えることも可能です。  
上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 
${x}$は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。  
分類問題であれば、それぞれの時刻の${h}$に対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。出力は最後の${h}$だけを使用する  場合と、全ての${h}$を使う場合があります。

In [16]:
class RNN:
    def __init__(self, w_x, w_h, b):
        self.params = [w_x, w_h, b]
        self.grads = [np.zeros_like(w_x), np.zeros_like(w_h), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        w_x, w_h, b = self.params
        t = np.dot(h_prev, w_h) + np.dot(x, w_x) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next):
        w_x, w_h, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dw_h = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt, w_h.T)
        dw_x = np.dot(x.T, dt)
        dx = np.dot(dt, w_x.T)

        self.grads[0][...] = dw_x
        self.grads[1][...] = dw_h
        self.grads[2][...] = db

        return dx, dh_prev
    
class SGD:
    '''
    確率的勾配降下法（Stochastic Gradient Descent）
    '''
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]





In [17]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh)
            dxs[:, t, :] = dx

            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None



In [18]:
class Softmax:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None

    def forward(self, x):
        self.out = softmax(x)
        return self.out

    def backward(self, dout):
        dx = self.out * dout
        sumdx = np.sum(dx, axis=1, keepdims=True)
        dx -= self.out * sumdx
        return dx


class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None  # softmaxの出力
        self.t = None  # 教師ラベル

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 教師ラベルがone-hotベクトルの場合、正解のインデックスに変換
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx


In [22]:
import numpy as np
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes))
b = np.array([1])
b_s = np.zeros((batch_size,n_sequences,n_features))
b_s = TimeRNN(w_x,w_h,b)
result_of_forward = b_s.forward(x)
print(result_of_forward[:,2,:])

[[0.79494226 0.81839    0.8393965  0.85584176]]
